In [17]:
import os
import time
import io

In [2]:
infile = 'giphy.gif'
os.path.exists(infile)

True

In [18]:
file_obj = open(infile, mode='rb')

In [ ]:
file_obj.close()

In [19]:
type(file_obj)

_io.BufferedReader

# Compare signature

In [4]:
sig = file_obj.read(6)
sig

b'GIF89a'

In [15]:
len(sig)

6

In [5]:
type(sig)

bytes

In [8]:
gif87a_sig = b'GIF87a'
gif89a_sig = b'GIF89a'

In [22]:
msg = f'Hi {str(gif89a_sig)}'
print(msg)

Hi b'GIF89a'


In [9]:
type(gif89a_sig)

bytes

In [11]:
gif89a_sig

b'GIF89a'

In [12]:
sig == gif89a_sig

True

In [14]:
sig == gif87a_sig

False